## 1. Configuración Inicial

Verificamos el ambiente de Databricks y las versiones de componentes.

In [ ]:
# Verificar versión de Spark
print(f"Spark Version: {spark.version}")
print(f"Python Version: {spark.conf.get('spark.executorEnv.PYTHONHASHSEED', 'Not set')}")

# Importar librerías necesarias
from pyspark.sql import functions as F
from pyspark.sql.types import *
from delta.tables import DeltaTable
from datetime import datetime, timedelta

print("\n✓ Librerías importadas correctamente")

## 2. Carga y Exploración de Datos

Cargamos el dataset de energía mundial (OWID) y exploramos su estructura.

In [ ]:
# Cargar datos desde CSV
df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("/workspaces/AzureDatabricksWorkshop/FundamentosArquitecturaAzureDatabricks/owid-energy-data.csv")

# Información básica
print(f"Total de registros: {df.count():,}")
print(f"Total de columnas: {len(df.columns)}")
print(f"\nPrimeras columnas: {df.columns[:10]}")

In [ ]:
# Mostrar esquema del DataFrame
df.printSchema()

In [ ]:
# Estadísticas descriptivas
display(df.select(
    "country", 
    "year", 
    "population", 
    "gdp", 
    "primary_energy_consumption"
).describe())

In [ ]:
# Muestra de datos
display(df.select(
    "country", 
    "year", 
    "population", 
    "gdp",
    "primary_energy_consumption",
    "renewables_consumption"
).limit(20))

## 3. Análisis de Calidad de Datos

Identificamos problemas de calidad: valores nulos, duplicados, outliers.

In [ ]:
# Análisis de valores nulos por columna
null_counts = df.select([
    F.count(F.when(F.col(c).isNull(), c)).alias(c) 
    for c in df.columns
])

# Transponer para mejor visualización
null_data = [(col, null_counts.select(col).first()[0]) for col in null_counts.columns]
null_df = spark.createDataFrame(null_data, ["column", "null_count"])
null_df_filtered = null_df.filter(F.col("null_count") > 0).orderBy(F.desc("null_count"))

display(null_df_filtered)

In [ ]:
# Detectar duplicados por país y año
duplicates = df.groupBy("country", "year").count().filter(F.col("count") > 1)
duplicate_count = duplicates.count()

print(f"Registros duplicados (country + year): {duplicate_count}")

if duplicate_count > 0:
    display(duplicates.orderBy(F.desc("count")).limit(10))

In [ ]:
# Rango de años en el dataset
year_stats = df.select(
    F.min("year").alias("min_year"),
    F.max("year").alias("max_year"),
    F.count(F.col("year").isNotNull()).alias("valid_years")
)

display(year_stats)

## 4. Limpieza de Datos con PySpark

Aplicamos transformaciones para limpiar y preparar los datos.

In [ ]:
# 1. Eliminar duplicados
df_clean = df.dropDuplicates(["country", "year"])

print(f"Registros originales: {df.count():,}")
print(f"Registros después de eliminar duplicados: {df_clean.count():,}")
print(f"Duplicados eliminados: {df.count() - df_clean.count():,}")

In [ ]:
# 2. Filtrar registros válidos
df_clean = df_clean.filter(
    (F.col("country").isNotNull()) & 
    (F.col("year").isNotNull()) &
    (F.col("year") >= 1900) &
    (F.col("year") <= 2025)
)

print(f"Registros válidos: {df_clean.count():,}")

In [ ]:
# 3. Imputación de valores nulos - Consumos energéticos con 0
energy_cols = [
    "primary_energy_consumption",
    "renewables_consumption",
    "fossil_fuel_consumption",
    "coal_consumption",
    "gas_consumption",
    "oil_consumption"
]

for col in energy_cols:
    if col in df_clean.columns:
        df_clean = df_clean.withColumn(
            col,
            F.when(F.col(col).isNull(), 0).otherwise(F.col(col))
        )

print(f"✓ Valores nulos imputados en {len([c for c in energy_cols if c in df_clean.columns])} columnas de energía")

In [ ]:
# 4. Imputación con Imputer de ML para variables demográficas
from pyspark.ml.feature import Imputer

demographic_cols = ["population", "gdp"]
output_cols = [f"{col}_imputed" for col in demographic_cols]

imputer = Imputer(
    inputCols=demographic_cols,
    outputCols=output_cols
).setStrategy("median")

# Fit y transform
imputer_model = imputer.fit(df_clean)
df_clean = imputer_model.transform(df_clean)

print("✓ Imputación con mediana completada para población y GDP")
display(df_clean.select("country", "year", "population", "population_imputed", "gdp", "gdp_imputed").limit(10))

In [ ]:
# 5. Normalizar nombres de países
df_clean = df_clean.withColumn(
    "country",
    F.trim(F.upper(F.col("country")))
)

# Verificar normalización
print("Muestra de países normalizados:")
display(df_clean.select("country").distinct().orderBy("country").limit(15))

## 5. Transformaciones y Feature Engineering

Creamos nuevas variables derivadas para análisis avanzado.

In [ ]:
# 1. Ratio de energías renovables
df_transformed = df_clean.withColumn(
    "renewable_ratio",
    F.when(F.col("primary_energy_consumption") > 0,
           F.col("renewables_consumption") / F.col("primary_energy_consumption")
    ).otherwise(0)
)

# 2. Energía per cápita
df_transformed = df_transformed.withColumn(
    "energy_per_capita",
    F.when(F.col("population_imputed") > 0,
           F.col("primary_energy_consumption") / F.col("population_imputed") * 1000000
    ).otherwise(0)
)

# 3. Intensidad energética (energía/GDP)
df_transformed = df_transformed.withColumn(
    "energy_intensity",
    F.when(F.col("gdp_imputed") > 0,
           F.col("primary_energy_consumption") / F.col("gdp_imputed")
    ).otherwise(0)
)

# 4. Ratio de combustibles fósiles
df_transformed = df_transformed.withColumn(
    "fossil_ratio",
    F.when(F.col("primary_energy_consumption") > 0,
           F.col("fossil_fuel_consumption") / F.col("primary_energy_consumption")
    ).otherwise(0)
)

print("✓ Features calculados: renewable_ratio, energy_per_capita, energy_intensity, fossil_ratio")

In [ ]:
# 5. Categorización temporal
df_transformed = df_transformed.withColumn(
    "decade",
    (F.floor(F.col("year") / 10) * 10).cast(IntegerType())
)

# 6. Niveles de consumo
df_transformed = df_transformed.withColumn(
    "consumption_level",
    F.when(F.col("energy_per_capita") >= 100, "High")
     .when(F.col("energy_per_capita") >= 50, "Medium")
     .when(F.col("energy_per_capita") > 0, "Low")
     .otherwise("Unknown")
)

print("✓ Categorías creadas: decade, consumption_level")

# Visualizar distribución de niveles de consumo
display(df_transformed.groupBy("consumption_level").count().orderBy(F.desc("count")))

In [ ]:
# Visualizar datos transformados
display(df_transformed.select(
    "country",
    "year",
    "decade",
    "renewable_ratio",
    "energy_per_capita",
    "energy_intensity",
    "consumption_level"
).orderBy(F.desc("energy_per_capita")).limit(20))

## 6. Agregaciones y Análisis

Creamos vistas agregadas para análisis temporal y por región.

In [ ]:
# Agregación por década
df_by_decade = df_transformed.groupBy("decade").agg(
    F.avg("primary_energy_consumption").alias("avg_energy_consumption"),
    F.avg("renewable_ratio").alias("avg_renewable_ratio"),
    F.avg("fossil_ratio").alias("avg_fossil_ratio"),
    F.sum("population_imputed").alias("total_population"),
    F.count("*").alias("record_count")
).orderBy("decade")

print("Tendencias por década:")
display(df_by_decade)

In [ ]:
# Top 10 países por consumo energético (últimos datos disponibles)
from pyspark.sql.window import Window

window_spec = Window.partitionBy("country").orderBy(F.desc("year"))

df_latest = df_transformed.withColumn(
    "row_num",
    F.row_number().over(window_spec)
).filter(F.col("row_num") == 1)

top_consumers = df_latest.select(
    "country",
    "year",
    "primary_energy_consumption",
    "renewable_ratio",
    "energy_per_capita"
).orderBy(F.desc("primary_energy_consumption")).limit(10)

print("Top 10 países por consumo energético total:")
display(top_consumers)

## 7. Validación de Calidad de Datos

Implementamos reglas de validación automatizadas.

In [ ]:
def validate_data_quality(df, rules):
    """
    Valida reglas de calidad en el DataFrame
    
    Args:
        df: DataFrame de Spark
        rules: Diccionario con reglas de validación
    
    Returns:
        DataFrame con resultados de validación
    """
    validation_results = []
    total = df.count()
    
    for rule_name, rule_condition in rules.items():
        passed = df.filter(rule_condition).count()
        failed = total - passed
        
        validation_results.append({
            "rule": rule_name,
            "total_records": total,
            "passed": passed,
            "failed": failed,
            "pass_rate": round((passed / total * 100) if total > 0 else 0, 2)
        })
    
    return spark.createDataFrame(validation_results)

# Definir reglas de validación
validation_rules = {
    "year_in_range": (F.col("year") >= 1900) & (F.col("year") <= 2025),
    "population_positive": F.col("population_imputed") >= 0,
    "energy_non_negative": F.col("primary_energy_consumption") >= 0,
    "renewable_ratio_valid": (F.col("renewable_ratio") >= 0) & (F.col("renewable_ratio") <= 1),
    "country_not_null": F.col("country").isNotNull(),
    "decade_consistent": F.col("decade") == (F.floor(F.col("year") / 10) * 10)
}

# Ejecutar validación
validation_report = validate_data_quality(df_transformed, validation_rules)

print("Reporte de Calidad de Datos:")
display(validation_report)

## 8. Delta Lake - Guardar Datos con ACID

Escribimos los datos transformados en formato Delta Lake.

In [ ]:
# Definir ruta para Delta Lake
delta_path = "/tmp/delta/energy_processed"

# Escribir datos como tabla Delta
df_transformed.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .save(delta_path)

print(f"✓ Tabla Delta creada en: {delta_path}")
print(f"✓ Registros guardados: {df_transformed.count():,}")

In [ ]:
# Verificar que podemos leer la tabla Delta
df_delta = spark.read.format("delta").load(delta_path)

print(f"Registros en Delta Lake: {df_delta.count():,}")
display(df_delta.limit(10))

## 9. Versionado y Time Travel

Exploramos las capacidades de versionado de Delta Lake.

In [ ]:
# Obtener tabla Delta
delta_table = DeltaTable.forPath(spark, delta_path)

# Ver historial de versiones
history = delta_table.history()

print("Historial de versiones:")
display(history.select("version", "timestamp", "operation", "operationMetrics"))

In [ ]:
# Hacer una actualización para crear nueva versión
delta_table.update(
    condition="year = 2020 AND country = 'UNITED STATES'",
    set={"consumption_level": "'Updated'"}
)

print("✓ Registros actualizados")

# Verificar nueva versión
history_updated = delta_table.history()
print(f"\nTotal de versiones: {history_updated.count()}")
display(history_updated.select("version", "timestamp", "operation").limit(5))

In [ ]:
# Time Travel: Leer versión anterior
df_version_0 = spark.read \
    .format("delta") \
    .option("versionAsOf", 0) \
    .load(delta_path)

df_version_1 = spark.read \
    .format("delta") \
    .option("versionAsOf", 1) \
    .load(delta_path)

print(f"Registros en versión 0: {df_version_0.count():,}")
print(f"Registros en versión 1: {df_version_1.count():,}")

# Comparar registros de Estados Unidos
print("\nComparación de versiones (USA, 2020):")
display(df_version_0.filter((F.col("country") == "UNITED STATES") & (F.col("year") == 2020)).select("country", "year", "consumption_level"))
display(df_version_1.filter((F.col("country") == "UNITED STATES") & (F.col("year") == 2020)).select("country", "year", "consumption_level"))

## 10. Operaciones ACID: Merge (Upsert)

Demostramos la operación MERGE para actualizar e insertar datos.

In [ ]:
# Crear datos nuevos para MERGE
new_data = spark.createDataFrame([
    ("BRAZIL", 2024, 215000000.0, 5000.0, 0.45, 23.36, 0.55, "High", 2020),
    ("INDIA", 2024, 1400000000.0, 6000.0, 0.35, 4.29, 0.65, "Low", 2020),
    ("GERMANY", 2024, 83000000.0, 3500.0, 0.50, 42.17, 0.50, "High", 2020)
], ["country", "year", "population_imputed", "primary_energy_consumption", 
    "renewable_ratio", "energy_per_capita", "fossil_ratio", "consumption_level", "decade"])

print("Datos nuevos a fusionar:")
display(new_data)

In [ ]:
# Ejecutar MERGE (Upsert)
delta_table.alias("target").merge(
    new_data.alias("source"),
    "target.country = source.country AND target.year = source.year"
).whenMatchedUpdateAll() \
 .whenNotMatchedInsertAll() \
 .execute()

print("✓ Operación MERGE completada")

# Verificar resultados
df_merged = spark.read.format("delta").load(delta_path)
print(f"\nTotal de registros después de MERGE: {df_merged.count():,}")

# Mostrar registros fusionados
display(df_merged.filter(
    (F.col("country").isin(["BRAZIL", "INDIA", "GERMANY"])) & 
    (F.col("year") == 2024)
))

## 11. Optimización con OPTIMIZE y Z-ORDER

Optimizamos la tabla Delta para mejorar performance de consultas.

In [ ]:
# OPTIMIZE: Compactar archivos pequeños
spark.sql(f"""
    OPTIMIZE delta.`{delta_path}`
""")

print("✓ OPTIMIZE completado")

In [ ]:
# Z-ORDER: Optimizar para queries frecuentes
spark.sql(f"""
    OPTIMIZE delta.`{delta_path}`
    ZORDER BY (country, year)
""")

print("✓ Z-ORDER completado en columnas: country, year")

In [ ]:
# Ver detalles de la tabla optimizada
detail_df = spark.sql(f"DESCRIBE DETAIL delta.`{delta_path}`")
display(detail_df.select("format", "numFiles", "sizeInBytes", "properties"))

## 12. Particionamiento

Creamos una tabla particionada para mejorar performance de queries con filtros.

In [ ]:
# Crear tabla particionada por década
delta_path_partitioned = "/tmp/delta/energy_partitioned"

df_transformed.write \
    .format("delta") \
    .mode("overwrite") \
    .partitionBy("decade") \
    .option("overwriteSchema", "true") \
    .save(delta_path_partitioned)

print(f"✓ Tabla particionada creada en: {delta_path_partitioned}")
print("✓ Particionada por: decade")

In [ ]:
# Verificar estructura de particiones
partitioned_detail = spark.sql(f"DESCRIBE DETAIL delta.`{delta_path_partitioned}`")
display(partitioned_detail.select("format", "partitionColumns", "numFiles", "sizeInBytes"))

## 13. Pipeline ETL Completo - Función Reutilizable

Encapsulamos todo el proceso ETL en una función.

In [ ]:
def etl_pipeline(source_path, target_path, partition_cols=None):
    """
    Pipeline ETL completo: Extract, Transform, Load
    
    Args:
        source_path: Ruta al archivo fuente
        target_path: Ruta de destino para Delta Table
        partition_cols: Lista de columnas para particionar
    
    Returns:
        Número de registros procesados
    """
    print("="*60)
    print("INICIANDO PIPELINE ETL")
    print("="*60)
    
    # EXTRACT
    print("\n[1/5] EXTRACT - Cargando datos...")
    df_raw = spark.read.format("csv") \
        .option("header", "true") \
        .option("inferSchema", "true") \
        .load(source_path)
    
    initial_count = df_raw.count()
    print(f"✓ Registros cargados: {initial_count:,}")
    
    # TRANSFORM - Limpieza
    print("\n[2/5] TRANSFORM - Limpiando datos...")
    df_clean = df_raw.dropDuplicates(["country", "year"]) \
        .filter((F.col("country").isNotNull()) & (F.col("year").isNotNull()))
    
    # Imputar valores nulos
    for col_name in ["population", "gdp", "primary_energy_consumption"]:
        if col_name in df_clean.columns:
            df_clean = df_clean.withColumn(
                col_name,
                F.when(F.col(col_name).isNull(), 0).otherwise(F.col(col_name))
            )
    
    clean_count = df_clean.count()
    print(f"✓ Registros después de limpieza: {clean_count:,} ({initial_count - clean_count:,} eliminados)")
    
    # TRANSFORM - Features
    print("\n[3/5] TRANSFORM - Creando features...")
    df_transformed = df_clean \
        .withColumn("renewable_ratio",
                   F.when(F.col("primary_energy_consumption") > 0,
                         F.col("renewables_consumption") / F.col("primary_energy_consumption"))
                   .otherwise(0)) \
        .withColumn("decade", (F.floor(F.col("year") / 10) * 10).cast(IntegerType())) \
        .withColumn("processing_timestamp", F.current_timestamp())
    
    print(f"✓ Features creados: renewable_ratio, decade, processing_timestamp")
    
    # LOAD
    print("\n[4/5] LOAD - Guardando en Delta Lake...")
    writer = df_transformed.write \
        .format("delta") \
        .mode("overwrite") \
        .option("overwriteSchema", "true")
    
    if partition_cols:
        writer = writer.partitionBy(*partition_cols)
        print(f"✓ Particionado por: {', '.join(partition_cols)}")
    
    writer.save(target_path)
    
    # OPTIMIZE
    print("\n[5/5] OPTIMIZE - Optimizando Delta Table...")
    spark.sql(f"OPTIMIZE delta.`{target_path}` ZORDER BY (country, year)")
    
    print("\n" + "="*60)
    print("PIPELINE ETL COMPLETADO CON ÉXITO")
    print("="*60)
    print(f"📊 Registros procesados: {clean_count:,}")
    print(f"📁 Ubicación: {target_path}")
    print(f"⚡ Optimizado con Z-ORDER")
    
    return clean_count

print("✓ Función ETL pipeline definida")

In [ ]:
# Ejecutar pipeline completo
records_processed = etl_pipeline(
    source_path="/workspaces/AzureDatabricksWorkshop/FundamentosArquitecturaAzureDatabricks/owid-energy-data.csv",
    target_path="/tmp/delta/energy_final",
    partition_cols=["decade"]
)

## 14. Monitoreo y Reporte de Salud

Generamos métricas y reportes de la tabla Delta.

In [ ]:
def monitor_delta_table(delta_path):
    """
    Genera reporte de salud de tabla Delta
    """
    delta_table = DeltaTable.forPath(spark, delta_path)
    df = spark.read.format("delta").load(delta_path)
    detail = spark.sql(f"DESCRIBE DETAIL delta.`{delta_path}`").first()
    
    report = {
        "total_records": df.count(),
        "total_columns": len(df.columns),
        "table_size_mb": round(detail["sizeInBytes"] / (1024*1024), 2),
        "number_of_files": detail["numFiles"],
        "latest_version": delta_table.history().select("version").first()[0]
    }
    
    print("="*60)
    print("REPORTE DE SALUD - DELTA TABLE")
    print("="*60)
    for key, value in report.items():
        print(f"{key.replace('_', ' ').title()}: {value:,}")
    print("="*60)
    
    return report

# Generar reporte
health_report = monitor_delta_table("/tmp/delta/energy_final")

## 15. Consultas SQL sobre Delta Lake

Demostramos queries SQL sobre las tablas Delta creadas.

In [ ]:
# Crear vista temporal para queries SQL
df_final = spark.read.format("delta").load("/tmp/delta/energy_final")
df_final.createOrReplaceTempView("energy_analysis")

print("✓ Vista temporal 'energy_analysis' creada")

In [ ]:
%%sql
-- Top 10 países con mayor ratio de renovables (2020)
SELECT 
    country,
    year,
    ROUND(renewable_ratio * 100, 2) as renewable_percentage,
    primary_energy_consumption
FROM energy_analysis
WHERE year = 2020
  AND primary_energy_consumption > 100
ORDER BY renewable_ratio DESC
LIMIT 10

In [ ]:
%%sql
-- Evolución promedio de renovables por década
SELECT 
    decade,
    ROUND(AVG(renewable_ratio) * 100, 2) as avg_renewable_pct,
    COUNT(*) as country_count
FROM energy_analysis
WHERE decade >= 1980
GROUP BY decade
ORDER BY decade

## 16. Conclusión y Resumen

¡Felicitaciones! Has completado el Lab 2 de Preparación y Procesamiento de Datos.

In [ ]:
# Resumen de lo aprendido
summary = {
    "Habilidades Adquiridas": [
        "✅ Exploración de datos con múltiples lenguajes",
        "✅ Ingesta eficiente con Auto Loader",
        "✅ Limpieza y transformación con PySpark",
        "✅ Implementación de Delta Lake con ACID",
        "✅ Versionado y Time Travel",
        "✅ Optimización con OPTIMIZE y Z-ORDER",
        "✅ Schema evolution y escalabilidad",
        "✅ Pipelines ETL reproducibles"
    ],
    "Archivos Creados": [
        "/tmp/delta/energy_processed",
        "/tmp/delta/energy_partitioned",
        "/tmp/delta/energy_final"
    ],
    "Próximos Pasos": [
        "Lab 3: Feature Engineering y Exploración de Datos",
        "Lab 4: Entrenamiento de Modelos con MLflow"
    ]
}

print("="*60)
print("LABORATORIO COMPLETADO")
print("="*60)

for section, items in summary.items():
    print(f"\n{section}:")
    for item in items:
        print(f"  {item}")

print("\n" + "="*60)
print("¡EXCELENTE TRABAJO!")
print("="*60)